#  Data analysis using SQLite for the reviews of fine foods from amazon, 
- In this report, I will analyze the reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.

- Data includes:

  - Reviews from Oct 1999 - Oct 2012
  - 568,454 reviews
  - 256,059 users
  - 74,258 products
  - 260 users with > 50 reviews
  - wordcloud

https://www.kaggle.com/snap/amazon-fine-food-reviews?select=database.sqlite

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
conn= sqlite3.connect('database.sqlite')
c=conn.cursor()

### Data Wrangeling

In [3]:
dataframe= pd.read_sql("""SELECT * FROM sqlite_master;""", conn)

In [4]:
dataframe

,type,name,tbl_name,rootpage,sql
0,table,Reviews,Reviews,2,CREATE TABLE Reviews (\n Id INTEGER PRIMARY...


In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
type        1 non-null object
name        1 non-null object
tbl_name    1 non-null object
rootpage    1 non-null int64
sql         1 non-null object
dtypes: int64(1), object(4)
memory usage: 168.0+ bytes


In [6]:
dataframe.describe()

,rootpage
count,1.0
mean,2.0
std,NaN
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0


In [7]:
df_table= pd.read_sql("""SELECT * FROM Reviews;""", conn)

In [8]:
df_table

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


### I would like to see how many product get Score 5.

In [9]:
pd.read_sql("""SELECT COUNT(Score) FROM Reviews
             WHERE Score>4 ORDER BY Score DESC ;""", conn)

,COUNT(Score)
0,363122


### What is the average score from reviewers for the food?

In [10]:
pd.read_sql("""SELECT AVG(Score) FROM Reviews ;""", conn)

,AVG(Score)
0,4.183199


In [11]:
# Select review where the score is not 3 for the specific user ID to see if we have duplicated data
pd.read_sql("""
SELECT * FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR" ORDER BY ProductID
""", conn)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


- You can see that the same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text.
- Often our results will include many duplicate values. If we want to select all the unique values from a column, we use the DISTINCT keyword 

In [12]:
pd.read_sql(""" SELECT DISTINCT Summary, ProfileName FROM Reviews
WHERE Score=5 LIMIT 10;""", conn)

,Summary,ProfileName
0,Good Quality Dog Food,delmartian
1,Great taffy,"Michael D. Bigham ""M. Wassir"""
2,Great! Just as good as the expensive brands!,David C. Sullivan
3,"Wonderful, tasty taffy",Pamela G. Williams
4,Yay Barley,R. James
5,Healthy Dog Food,Carol A. Reed
6,The Best Hot Sauce in the World,Canadian Fan
7,"My cats LOVE this ""diet"" food better than thei...","A Poeng ""SparkyGoHome"""
8,Strawberry Twizzlers - Yummy,"Lynrie ""Oh HELL no"""
9,"Lots of twizzlers, just what you expect.",Brian A. Lee


### get the schema of the table


In [13]:
pd.read_sql("PRAGMA table_info('Reviews');",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,0,None,1
1,1,ProductId,TEXT,0,None,0
2,2,UserId,TEXT,0,None,0
3,3,ProfileName,TEXT,0,None,0
4,4,HelpfulnessNumerator,INTEGER,0,None,0
5,5,HelpfulnessDenominator,INTEGER,0,None,0
6,6,Score,INTEGER,0,None,0
7,7,Time,INTEGER,0,None,0
8,8,Summary,TEXT,0,None,0
9,9,Text,TEXT,0,None,0


In [14]:
pd.read_sql(""" SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", conn)

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2
...,...,...,...,...,...,...,...
80663,AZZNK89PXD006,B001J3ZUQ6,Etude,1269648000,5,In this package you get three varieties of pop...,6
80664,AZZTH6DJ0KSIP,B00014IVPQ,Crystal Caccamo,1304208000,5,ive bought a few different kinds and i feel th...,3
80665,AZZU1VEO8KUXH,B003FDC2I2,"Mia P ""Mia P""",1317513600,5,I bought this for my 13 year old daughter who ...,3
80666,AZZU4D6TZ2L6J,B0029XITW2,"Sherry King ""llamasmama""",1247875200,5,My father thought this was the best cheese eve...,2


In [15]:
pd.read_sql(""" SELECT  UserId, COUNT(ProductId), ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews 
""", conn)

,UserId,COUNT(ProductId),ProfileName,Time,Score,Text,COUNT(*)
0,A3SGXH7AUHU8GW,568454,delmartian,1303862400,5,I have bought several of the Vitality canned d...,568454


In [16]:
pd.read_sql(""" SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
WHERE ProfileName='Penguin Chick' 
ORDER BY Time;
""", conn)

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,A3FBM0RMCMAABV,B007K449CE,Penguin Chick,1345507200,5,I'm really not sure why the company decided to...,20


In [17]:
pd.read_sql(""" SELECT 
MIN(Score) As min_score,
MAX(Score) AS max_score,
AVG(Score) AS avg_score
FROM Reviews;""", conn)

,min_score,max_score,avg_score
0,1,5,4.183199


### Create a specific index and get the details about  it

In [18]:
c.execute("""CREATE INDEX idx_ProfileName
ON Reviews (ProfileName);""")

In [19]:
pd.read_sql("""PRAGMA index_info('idx_ProfileName');""",conn)

,seqno,cid,name
0,0,3,ProfileName


In [28]:
pd.read_sql("""SELECT strftime(TIME), Score FROM Reviews
ORDER BY Score DESC;""", conn)

,strftime(TIME),Score
0,1303862400,5
1,1350777600,5
2,1340150400,5
3,1336003200,5
4,1322006400,5
...,...,...
568449,1187049600,1
568450,1341532800,1
568451,1319673600,1
568452,1319500800,1


In [29]:
pd.read_sql("""SELECT Datetime(TIME), Score FROM Reviews
ORDER BY Score DESC;""", conn)

,Datetime(TIME),Score
0,None,5
1,None,5
2,None,5
3,None,5
4,None,5
...,...,...
568449,None,1
568450,None,1
568451,None,1
568452,None,1


In [33]:
df_table['Date_Time']= pd.to_datetime(df_table['Time'], unit='s')

In [34]:
df_table.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Date_Time
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,2011-04-27
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-07
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,2008-08-18
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,2011-06-13
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-21
